In [ ]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
words_to_translate = df_text


# Function to translate a word using ChatGPT API
def translate_word(word):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German words to English."},
                {"role": "user", "content": f'Translate the following German word to English: "{word}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        # print(response.choices[0].message.content.strip())
        
        #content = response.choices[0].message.content.strip()
        #start_index = content.find('is "') + 4
        #end_index = content.find('"', start_index)
        #translation = content[start_index:end_index]
        #print(translation)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating word '{word}': {e}")
        return None

# Step 2: Translate the words
translations = [translate_word(word) for word in words_to_translate]

# Remove None values if any translation fails
translated_words = [word for word, translation in zip(words_to_translate, translations) if translation is not None]
translations = [translation for translation in translations if translation is not None]

# Step 3: Store the translations in a .xlsx file
df_translated_text = pd.DataFrame({
    'German': translated_words,
    'English': translations
})
print(df_translated_text)

In [ ]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
comments_to_translate = df_comment['comment']
comments_ids_to_translate = df_comment['id']

# Function to translate a word using ChatGPT API
def translate_comment(comment):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German comments to English."},
                {"role": "user", "content": f'Translate the following German comment to English: "{comment}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating comment '{comment}': {e}")
        return None

# Step 2: Translate the words with logging and saving
translations = []
counter = 0
save_interval = 50
print_interval = 100

for comment in comments_to_translate:
    translation = translate_comment(comment)
    if translation:
        translations.append(translation)
    counter += 1
    if counter % print_interval == 0:
        print(f"Current counter: {counter}")
    if counter % save_interval == 0:
        df_temp = pd.DataFrame({'ID': comments_ids_to_translate[:counter], 'German': comments_to_translate[:counter], 'English': translations})
        df_temp.to_excel(f'translations_comments_up_to_{counter}.xlsx', index=False)

# Final save
df_translated_comments = pd.DataFrame({
    'ID': comments_ids_to_translate,
    'German': comments_to_translate,
    'English': translations
})
df_translated_comments.to_excel('translations_comments.xlsx', index=False)

print(df_translated_comments)
